<a href="https://colab.research.google.com/github/SVT1-2/lightweight-charts-python/blob/main/m3u8_downl_SvT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://github.com/jakiyaa/m3u8-downloader">
<img src="https://github.com/fluidicon.png" height=100px, align="left"></a>



<center><h1><b>M3u8 online Downloader </h1></b></center>

In [ ]:
#@title ← Run <Center>Mount Google Drive </Center>
#@markdown <Center> <img src="https://kstatic.googleusercontent.com/files/e20dbc1db7b6eab22fb2d07df8043f8a74b301dbb8c176e6a1992b2dc229c5ae94b5c260cf549c41712fa40b7e639166c6a8461303a27c2a3b9c55b8ca155ec3" class="icon" alt="Icon">

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title ← Run <center> </font> <font color='red'> Paste m3u8 link <img src='https://play-lh.googleusercontent.com/vi_F_8raK3dL9D32R1VJRmWv18VKvc3niKDOE5z6yQsHOyzH0FQ1vq91eeakoEIFuxH9' height="40" /> </center> { display-mode: "form" }
import os
M3u8_link = "" #@param {type:"string"}
Download_location = "/content/drive/MyDrive/myvideo.mp4" #@param {type:"string"}



!sudo apt install -y ffmpeg
!pip install --user m3u8downloader

!~/.local/bin/downloadm3u8 -o "$Download_location" "$M3u8_link"



In [ ]:
# @title 📺 Precise Fragment Downloader
m3u8_url = "" # @param {type:"string"}
file_name = "downloaded_video52" # @param {type:"string"}
auto_download_to_pc = False # @param {type:"boolean"}

import os
import re
import subprocess
from tqdm.notebook import tqdm
from google.colab import files

def get_segment_count(url):
    try:
        # Check how many segments are in the playlist
        result = subprocess.run(['curl', '-s', url], capture_output=True, text=True)
        count = len(re.findall(r'#EXTINF', result.stdout))
        return count if count > 0 else 100 # Default to 100 if count fails
    except:
        return 100

# Setup names
output_file = file_name if file_name.endswith('.mp4') else f"{file_name}.mp4"
total_segments = get_segment_count(m3u8_url)

if m3u8_url:
    print(f"📦 Total fragments detected: {total_segments}")

    # Corrected Command List (strings used for all arguments)
    cmd = [
        'ffmpeg',
        '-headers', 'Referer: https://www.pornhub.com/',
        '-user_agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
        '-i', m3u8_url,
        '-c', 'copy',
        '-y',
        output_file
    ]

    # Start process and monitor fragments via stderr
    process = subprocess.Popen(cmd, stderr=subprocess.PIPE, universal_newlines=True)

    pbar = tqdm(total=total_segments, unit="frag", desc="Fetching")

    for line in process.stderr:
        # FFmpeg logs 'Opening... for reading' for every HLS fragment it touches
        if "Opening" in line and "for reading" in line:
            pbar.update(1)

    process.wait()
    pbar.close()

    if os.path.exists(output_file) and os.path.getsize(output_file) > 0:
        print(f"\n✅ All fragments merged into: {output_file}")
        if auto_download_to_pc:
            files.download(output_file)
    else:
        print("\n❌ Error: Download failed. Your link likely expired while the script was running.")
else:
    print("⚠️ Please enter a URL.")